In [50]:
# Imports and functions
import os
import netCDF4 as nc4
import numpy as np
%matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


In [51]:
# User inputs
#indir = '/home/disk/bob/gpm/nht_ku/classify/ex_data_v05'
indir = '/home/disk/bob/gpm/nht_ku/classify/ex_data_ite132'
iyear = '2015'
imonth = '08'
VERT_RES = 0.125  #km
STRA = 1
CONV = 2
THRESHOLD = 0.0  #dBZ
DELTA_HT_FOR_SLOPE = 1.5  #km
SLOPE = 7.5
MAX_BB_HT = 5.0   #km
REFL_MAX_OFFSET = 2.0 #dBZ

fname = 'GPM2Ku5_uw3_20150808.020818_to_20150808.021247_008190_NHW.nc'
print fname

GPM2Ku5_uw3_20150808.020818_to_20150808.021247_008190_NHW.nc


In [52]:
# Read input file
ncid = nc4.Dataset(indir+'/'+iyear+'/'+imonth+'/'+fname,'r')
refl = np.array(ncid.variables['refl'])
(ntime,nalt,nlat,nlon) = refl.shape
refl_missing = ncid.variables['refl']._FillValue
refl[:,0,:,:] = refl_missing  # get rid of lowest level due to suspicious data
bin_missing = int(refl_missing)

rt = np.array(ncid.variables['rain_type'])
rt_missing = ncid.variables['rain_type']._FillValue

rt_uw = np.array(rt, copy=True)

srt = np.array(ncid.variables['shallow_rain_type'])
srt_missing = ncid.variables['shallow_rain_type']._FillValue

In [53]:
# Find max_refl and max_refl_ht at each lat/lon
max_refl = np.zeros(rt.shape,dtype=float)+refl_missing
max_refl = np.amax(refl,axis=1)
max_refl_bin = np.zeros(rt.shape,dtype=int)+bin_missing
max_refl_bin = np.argmax(refl,axis=1)
max_refl_ht = max_refl_bin * VERT_RES

max_refl_bin[(max_refl==refl_missing)] = bin_missing
max_refl_ht[(max_refl==refl_missing)] = refl_missing

In [54]:
# initialize reference refl/bin/ht
# these reflect highest refl,bin,ht above max_refl,bin.ht and within +/- REFL_MAX_OFFSET
reference_refl = np.array(max_refl, copy=True)
reference_refl_bin = np.array(max_refl_bin, copy=True)
reference_refl_ht = np.array(max_refl_ht, copy=True)

In [55]:
# find bin & height of highest good refl at each lat/lon
max_ht_bin = np.zeros(rt.shape,dtype=int) + bin_missing
for lev in range(nalt-1,-1,-1):
    max_ht_bin = np.where(np.logical_and((refl[:,lev,:,:] != refl_missing),
                                         (max_ht_bin == bin_missing) ), 
                          lev, max_ht_bin )
max_ht = max_ht_bin * (max_ht_bin != bin_missing) * VERT_RES
max_ht[(max_ht_bin==bin_missing)] = refl_missing

In [56]:
 # For rt_uw, change rain_type to stratiform if:
# 1. rain_type = convective and
# 2. shallow_rain_type = 0 and
# 3. max_refl in column >= THRESHOLD and   ## DON'T USE THIS CRITERIA ANYMORE
# 4. [(dbz @ reference point) - (dbz @ point 1.5km higher)]/
#    [(ht  @ reference point) - (ht  @ point 1.5km higher)] >= abs(SLOPE)
#  as long as ht @ reference point <= MAX_BB_HT   

In [57]:
# Determine indices of interest 
change = np.logical_and(rt==2,srt==0)
ind_time,ind_lat,ind_lon = np.where(change==True)
npixels = ind_time.size

In [58]:
# for each index
#    start at max_refl_ht & go up to find reference refl within +/- REFL_MAX_OFFSET
#    if height of new reference refl is less than MAX_BB_HT
#        determine slope between that point and point DELTA_HT_FOR_SLOPE higher or highest good point in column
#        if slope exceeds SLOPE, change raintype from convective to stratiform  

In [59]:
print('rt[0,132,69]=',rt[0,132,69])
print('rt[0,132,72]=',rt[0,132,72])
print('rt[0,98,52]=',rt[0,98,52])
print('rt[0,84,58]=',rt[0,84,58])
print('rt[0,130,63]=',rt[0,130,63])
print('rt[0,96,48]=',rt[0,96,48])
print('rt[0,83,75]=',rt[0,83,75])     

('rt[0,132,69]=', 1.0)
('rt[0,132,72]=', 1.0)
('rt[0,98,52]=', 1.0)
('rt[0,84,58]=', 1.0)
('rt[0,130,63]=', 2.0)
('rt[0,96,48]=', 2.0)
('rt[0,83,75]=', 2.0)


In [60]:
#for ind in range(0,npixels):
#itime = ind_time[ind]
#ilat = ind_lat[ind]
#ilon = ind_lon[ind]
# 20150808 case 1 - ite reclassified to stra
#itime = 0
#ilat = 132
#ilon = 69
# 20150808 case 2 - ite reclassified to stra
#itime = 0
#ilat = 132
#ilon = 72
# 20150808 case 3 - ite reclassified to stra
#itime = 0
#ilat = 98
#ilon = 52
# 20150808 case 4 - ite reclassified to stra
#itime = 0
#ilat = 84
#ilon = 58
# 20150808 case 5 - ite did not reclassify
itime = 0
ilat = 130
ilon = 63
# 20150808 case 6 - ite did not reclassify
#itime = 0
#ilat = 96
#ilon = 48
# 20150808 case 7 - ite did not reclassify
#itime = 0
#ilat = 83
#ilon = 75
if max_refl[itime,ilat,ilon] != refl_missing:
    next_bin = reference_refl_bin[itime,ilat,ilon]+1
    next_ht = reference_refl_ht[itime,ilat,ilon]+VERT_RES
    next_refl = refl[itime,next_bin,ilat,ilon]
    while abs(max_refl[itime,ilat,ilon]-next_refl) <= REFL_MAX_OFFSET:
        reference_refl[itime,ilat,ilon]=next_refl
        reference_refl_bin[itime,ilat,ilon] = next_bin
        reference_refl_ht[itime,ilat,ilon]=next_ht
        next_bin = reference_refl_bin[itime,ilat,ilon]+1
        next_ht = reference_refl_ht[itime,ilat,ilon]+VERT_RES
        next_refl = refl[itime,next_bin,ilat,ilon]
    if reference_refl_ht[itime,ilat,ilon] < MAX_BB_HT:
        if (reference_refl_ht[itime,ilat,ilon] + DELTA_HT_FOR_SLOPE) <= max_ht[itime,ilat,ilon]:
            compare_bin = reference_refl_bin[itime,ilat,ilon] + int(DELTA_HT_FOR_SLOPE/VERT_RES)
            compare_ht = reference_refl_ht[itime,ilat,ilon] + DELTA_HT_FOR_SLOPE
            compare_refl = refl[itime,compare_bin,ilat,ilon]
        else:
            compare_bin = max_ht_bin[itime,ilat,ilon]
            compare_ht = max_ht[itime,ilat,ilon]
            compare_refl = refl[itime,compare_bin,ilat,ilon]
        if (abs(reference_refl_ht[itime,ilat,ilon] - compare_ht)) != 0.0:
            if ( abs(reference_refl[itime,ilat,ilon] - compare_refl)/
                 abs(reference_refl_ht[itime,ilat,ilon] - compare_ht) ) >= SLOPE:
                rt_uw[itime,ilat,ilon] = STRA

In [61]:
# PLOT COLUMN
#some computations
delta_refl = reference_refl[itime,ilat,ilon] - compare_refl
delta_ht = compare_ht - reference_refl_ht[itime,ilat,ilon]
slope = delta_refl/delta_ht

#y = range(1, 30)
x = refl[0,0:54,ilat,ilon]
y = np.linspace(0.0,6.625,num=54)
fig = plt.figure()
plt.plot(x, y, '-',color='black')
#plt.plot(x, y, 'o')
#plt.xlim(15, 50);
plt.xlim(15, 60);
plt.ylim(0.25, 7.0);
plt.xlabel('refl (dBZ)')
plt.ylabel('ht (km)')

x1 = range(15,61)
y1 = np.array([MAX_BB_HT for i in xrange(len(x1))])
#plt.plot(x1, y1, 'r-',label='MAX_BB_HT') 
#plt.plot()

x2 = range(15,61)
y2 = np.array([max_refl_ht[itime,ilat,ilon] for i in xrange(len(x2))])
plt.plot(x2, y2, 'm-', label='max refl pt') 
plt.plot()

x3 = range(15,61)
y3 = np.array([reference_refl_ht[itime,ilat,ilon] for i in xrange(len(x3))])
plt.plot(x3, y3, 'b--', label='ref refl pt') 
plt.plot()

x4 = range(15,61)
y4 = np.array([compare_ht for i in xrange(len(x4))])
plt.plot(x4, y4, 'g--', label='compare refl pt') 
plt.plot()

y5 = range(0,8)
x5 = np.array([reference_refl[itime,ilat,ilon] for i in xrange(len(y5))])
plt.plot(x5, y5, 'b--') 
plt.plot()

y6 = range(0,8)
x6 = np.array([compare_refl for i in xrange(len(y6))])
plt.plot(x6, y6, 'g--') 
plt.plot()

plt.legend()

plt.fill_between(x1,y1,color='gray',alpha=0.3)

#plt.title('Reclassified Stratiform Reflectivity Profile - 20150808/020818')
plt.title('Convective Reflectivity Profile - 20150808/020818')

plt.text(18,reference_refl_ht[itime,ilat,ilon]+delta_ht/2,'Delta_ht='+str(round(delta_ht,2)))
plt.text(compare_refl+delta_refl/2,5,'Delta_refl='+str(round(delta_refl,2)),rotation=90)
plt.text(18,0.5,'SLOPE='+str(round(delta_refl/delta_ht,2)))

In [ ]:
# Replace old UW raintype field with new one in input netcdf file
ncid.variables['rain_type_uw'][:] = rt_uw

In [49]:
ncid.close()